In [53]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import torch.nn.functional as F

In [54]:
import torchvision.datasets as dsets
import torchvision.transforms as transforms

In [55]:
import matplotlib.pyplot as plt
import numpy as np

In [56]:
%matplotlib inline

In [57]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [58]:
image_size = 28 # the size of image is 28*28
num_classes = 10 # the number of all classes
num_epochs = 20 # the number of all epoches
batch_size = 64 # a mini batch,64 images
learning_rate=0.001

In [59]:
train_dataset = dsets.MNIST(root = './data',
                            train=True,
                            transform=transforms.ToTensor(),
                            download=True)

In [60]:
test_dataset = dsets.MNIST(root='./data',
                            train=False,
                            transform=transforms.ToTensor())

In [61]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                            batch_size=batch_size,
                                            shuffle=True)

In [62]:
indices = range(len(test_dataset))
indices_val = indices[:5000]
indices_test = indices[5000:]

In [63]:
sampler_val = torch.utils.data.sampler.SubsetRandomSampler(indices_val)
sampler_test = torch.utils.data.sampler.SubsetRandomSampler(indices_test)

In [64]:
validation_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                                batch_size = batch_size,
                                                shuffle = False,
                                                sampler = sampler_val)
test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                          batch_size = batch_size,
                                          shuffle= False,
                                          sampler=sampler_test)                                                

In [65]:
class ConvNet(nn.Module):
    def __init__(self,num_classes=10):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1,16,kernel_size=5,stride=1,padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(16,32,kernel_size=5,stride=1,padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2)
        )
        self.fc = nn.Linear(7*7*32,num_classes)
    
    def forward(self,x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0),-1)
        out = self.fc(out)
        return out

In [66]:
model = ConvNet(num_classes).to(device)

In [67]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)

In [68]:
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i,(images,labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        outputs =model(images)
        loss = criterion(outputs,labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1)%100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss:{:.4f}'.format(epoch+1,num_epochs,i+1,total_step,loss.item()))

Epoch [1/20], Step [100/938], Loss:0.2148
Epoch [1/20], Step [200/938], Loss:0.1020
Epoch [1/20], Step [300/938], Loss:0.1358
Epoch [1/20], Step [400/938], Loss:0.0711
Epoch [1/20], Step [500/938], Loss:0.1722
Epoch [1/20], Step [600/938], Loss:0.1042
Epoch [1/20], Step [700/938], Loss:0.1160
Epoch [1/20], Step [800/938], Loss:0.0417
Epoch [1/20], Step [900/938], Loss:0.0238
Epoch [2/20], Step [100/938], Loss:0.0121
Epoch [2/20], Step [200/938], Loss:0.0348
Epoch [2/20], Step [300/938], Loss:0.0325
Epoch [2/20], Step [400/938], Loss:0.0965
Epoch [2/20], Step [500/938], Loss:0.0580
Epoch [2/20], Step [600/938], Loss:0.0204
Epoch [2/20], Step [700/938], Loss:0.0162
Epoch [2/20], Step [800/938], Loss:0.0928
Epoch [2/20], Step [900/938], Loss:0.0175
Epoch [3/20], Step [100/938], Loss:0.0054
Epoch [3/20], Step [200/938], Loss:0.0477
Epoch [3/20], Step [300/938], Loss:0.0178
Epoch [3/20], Step [400/938], Loss:0.0244
Epoch [3/20], Step [500/938], Loss:0.0016
Epoch [3/20], Step [600/938], Loss

In [69]:
model.eval()

ConvNet(
  (layer1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Linear(in_features=1568, out_features=10, bias=True)
)

In [70]:
with torch.no_grad():
    correct = 0
    total = 0
    for images,labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data,1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100*correct/total))

Test Accuracy of the model on the 10000 test images: 99.6 %


In [71]:
torch.save(model.state_dict(),'model.ckpt')